In [2]:
!pip install pandas numpy scikit-learn xgboost imbalanced-learn streamlit shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.1 MB/s eta 0:00:00


In [13]:
from google.colab import files
uploaded = files.upload()

import pandas as pd

# Automatically read first uploaded CSV
filename = list(uploaded.keys())[0]

# Check file extension and read accordingly
if filename.endswith('.csv'):
    df = pd.read_csv(filename)
elif filename.endswith(('.xls', '.xlsx')):
    df = pd.read_excel(filename)
else:
    raise ValueError("Unsupported file format. Please upload a CSV or Excel file.")

print("Dataset Loaded:", filename)
df.head()

Saving vitD_donnees_2022Mai.xlsx to vitD_donnees_2022Mai (2).xlsx
Dataset Loaded: vitD_donnees_2022Mai (2).xlsx


,SOPK,Age,Testosterone,androstenedione,SDHEA,Waist circumference,BMI,FSH,LH,PRL,...,E2,@17OHprog,systolic blood pressure,diastolic blood pressure,GLU,total cholesterol,triglycerides,HDLc,LDLc,@25OHVitD
0,1,33.0,0.10,1.20,0.65,NaN,32.421875,3.6,4.5,30.0,...,124.5,1.45,NaN,NaN,5.99,3.53,1.29,1.40,1.500000,3.76
1,1,26.0,0.27,NaN,NaN,124.0,37.180250,6.7,14.7,18.2,...,NaN,NaN,100.0,80.0,5.32,4.38,0.91,1.07,2.896364,13.00
2,1,NaN,0.12,1.65,2.60,95.0,30.457656,7.7,6.0,14.8,...,46.6,0.90,130.0,70.0,4.80,4.90,0.95,0.87,2.900000,3.82
3,1,NaN,0.12,1.80,2.50,93.0,24.464602,6.4,3.1,10.0,...,47.6,0.60,110.0,80.0,5.30,4.48,1.48,0.97,2.800000,4.74
4,1,25.0,0.46,3.40,2.45,NaN,27.239224,6.2,7.4,7.6,...,16.0,1.00,120.0,80.0,4.09,5.17,1.95,0.84,3.443636,2.99


In [15]:
# Set your target column name
target_col = "SOPK"   # change if needed

# Convert Yes/No labels → 1/0 if necessary
if df[target_col].dtype == 'object':
    df[target_col] = df[target_col].map({'Y':1, 'N':0, 'Yes':1, 'No':0}).fillna(df[target_col]).astype(int)

print(df[target_col].value_counts())

SOPK
1    45
0    43
Name: count, dtype: int64


In [16]:
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from imblearn.over_sampling import SMOTE

In [17]:
# Identify numeric & categorical columns
numeric_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
numeric_cols.remove(target_col)

cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Preprocessing pipelines
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, numeric_cols),
    ('cat', cat_pipeline, cat_cols)
])

# Train-test split
X = df.drop(columns=[target_col])
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Fit preprocessor
X_train_p = preprocessor.fit_transform(X_train)
X_test_p = preprocessor.transform(X_test)

# Apply SMOTE for balancing
sm = SMOTE(random_state=42, k_neighbors=2)
X_train_smote, y_train_smote = sm.fit_resample(X_train_p, y_train)

print("Before SMOTE:", y_train.value_counts().to_dict())
print("After SMOTE:", pd.Series(y_train_smote).value_counts().to_dict())

Before SMOTE: {1: 36, 0: 34}
After SMOTE: {0: 36, 1: 36}


In [18]:
rf = RandomForestClassifier(
    n_estimators=200, class_weight='balanced', random_state=42
)
rf.fit(X_train_smote, y_train_smote)

xgb = XGBClassifier(
    eval_metric='logloss',
    random_state=42
)
xgb.fit(X_train_smote, y_train_smote)

print("Models trained successfully.")

Models trained successfully.


In [19]:
def evaluate(model, X_test, y_test):
    preds = model.predict(X_test)
    prob = model.predict_proba(X_test)[:,1]

    print("\nClassification Report:\n", classification_report(y_test, preds))
    print("Confusion Matrix:\n", confusion_matrix(y_test, preds))
    print("ROC AUC:", roc_auc_score(y_test, prob))

print("=== RANDOM FOREST ===")
evaluate(rf, X_test_p, y_test)

print("\n=== XGBOOST ===")
evaluate(xgb, X_test_p, y_test)

=== RANDOM FOREST ===

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.89      0.94         9
           1       0.90      1.00      0.95         9

    accuracy                           0.94        18
   macro avg       0.95      0.94      0.94        18
weighted avg       0.95      0.94      0.94        18

Confusion Matrix:
 [[8 1]
 [0 9]]
ROC AUC: 0.9876543209876543

=== XGBOOST ===

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.89      0.94         9
           1       0.90      1.00      0.95         9

    accuracy                           0.94        18
   macro avg       0.95      0.94      0.94        18
weighted avg       0.95      0.94      0.94        18

Confusion Matrix:
 [[8 1]
 [0 9]]
ROC AUC: 0.9876543209876543


In [20]:
best_model = xgb   # change to rf if random forest performs better
print("Selected Model:", type(best_model).__name__)

Selected Model: XGBClassifier


In [21]:
with open("pcos_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

with open("preprocessor.pkl", "wb") as f:
    pickle.dump(preprocessor, f)

print("Saved pcos_model.pkl and preprocessor.pkl")

Saved pcos_model.pkl and preprocessor.pkl


In [22]:
from google.colab import files
files.download("pcos_model.pkl")
files.download("preprocessor.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
files.download("requirements.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
%%writefile app.py
import streamlit as st
import pandas as pd
import pickle

# Load artifacts
preprocessor = pickle.load(open("preprocessor.pkl", "rb"))
model = pickle.load(open("pcos_model.pkl", "rb"))

st.title("PCOS Early Detection App")

st.write("Upload patient data for prediction:")

uploaded = st.file_uploader("Upload CSV", type=["csv"])

if uploaded:
    df = pd.read_csv(uploaded)
    X_p = preprocessor.transform(df)
    preds = model.predict(X_p)
    probs = model.predict_proba(X_p)[:,1]

    df["pcos_pred"] = preds
    df["pcos_prob"] = probs

    st.write(df)

    csv = df.to_csv(index=False)
    st.download_button("Download predictions", csv, "pcos_predictions.csv")
else:
    st.info("Upload a CSV file to begin.")

Writing app.py


In [26]:
files.download("app.py")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>